###Instalación

In [ ]:
pip install altair

In [ ]:
import warnings

# Ignorar advertencias de tipo FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.display import HTML

HTML('<link href="https://fonts.googleapis.com/css2?family=Lato:ital,wght@0,100;0,300;0,400;0,700;0,900;1,100;1,300;1,400;1,700;1,900&display=swap" rel="stylesheet">')

###Importar

In [ ]:
import altair as alt
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import IFrame

In [ ]:
# Cargar los datos desde el CSV
dn= '/content/drive/MyDrive/AAVV X COMUNA.csv'  # Reemplaza esto con la ruta correcta
data = pd.read_csv(dn)
print(data.head())

            Comuna  Total cantidad personas x comuna proyeccion 2021  \
0             BUIN                                            111934   
1  CALERA DE TANGO                                             29019   
2        CERRILLOS                                             89520   
3      CERRO NAVIA                                            142304   
4           COLINA                                            185599   

   Ninos (0-15) x comuna proyeccion 2021  \
0                                  23953   
1                                   5835   
2                                  17299   
3                                  26810   
4                                  47279   

   Cantidad de superficie comuna X m2 2021   superficie de AAVV  2018  \
0                                214000000                  603984.88   
1                                 73300000                   29946.03   
2                                 21000000                 1176964.65   
3         

###Explorar datos

In [ ]:
# Verificar los nombres de las columnas
print(data.columns)

Index(['Comuna', 'Total cantidad personas x comuna proyeccion 2021',
       'Ninos (0-15) x comuna proyeccion 2021',
       'Cantidad de superficie comuna X m2 2021', ' superficie de AAVV  2018',
       'Superficie AAVVm2 por cantidad habitante (0-15 anos)'],
      dtype='object')


In [ ]:
# Renombrar columnas para facilitar su uso
data.columns = [
    'Comuna',
    'Total_personas_2021',
    'Ninos_0_15_2021',
    'Superficie_comuna_m2_2021',
    'Superficie_AAVV_2018',  # Supongo que esta columna representa la superficie neta de áreas verdes
    'Superficie_AAVVm2_por_habitante'
]
# Mostrar las primeras filas para verificar los cambios
print(data.head())

            Comuna  Total_personas_2021  Ninos_0_15_2021  \
0             BUIN               111934            23953   
1  CALERA DE TANGO                29019             5835   
2        CERRILLOS                89520            17299   
3      CERRO NAVIA               142304            26810   
4           COLINA               185599            47279   

   Superficie_comuna_m2_2021  Superficie_AAVV_2018  \
0                  214000000             603984.88   
1                   73300000              29946.03   
2                   21000000            1176964.65   
3                   11000000             786372.48   
4                  971200000            1115239.62   

   Superficie_AAVVm2_por_habitante  
0                        25.215417  
1                         5.132139  
2                        68.036571  
3                        29.331312  
4                        23.588477  


###Calcular tasa

In [ ]:
# Calcular la tasa de superficie de áreas verdes por menor de edad
data['Tasa_Areas_Verdes'] = data['Superficie_AAVV_2018'] / data['Ninos_0_15_2021']

###Gráfico

In [ ]:
# Normalizar el tamaño de los puntos (más grande indica mejor tasa)
data['Tamaño_Punto'] = (data['Tasa_Areas_Verdes'] - data['Tasa_Areas_Verdes'].min()) / (data['Tasa_Areas_Verdes'].max() - data['Tasa_Areas_Verdes'].min()) * 500
data['Tamaño_Punto'] = data['Tamaño_Punto'].clip(lower=50)

# Crear un campo para resaltar las comunas específicas
comunas_destacadas = ['PUENTE ALTO', 'MAIPU']
data['Color'] = data['Comuna'].apply(lambda x: x.strip().upper() if x.strip().upper() in comunas_destacadas else 'Otras Comunas')

# Crear el gráfico de dispersión
chart = alt.Chart(data).mark_circle().encode(
    x='Ninos_0_15_2021',  # Eje X: Cantidad de Niños
    y='Superficie_AAVV_2018',  # Eje Y: Superficie neta de áreas verdes
    size=alt.Size('Tamaño_Punto', legend=None),  # Tamaño de los círculos
    color=alt.Color('Color:N',
                    scale=alt.Scale(domain=comunas_destacadas + ['Otras Comunas'],
                                    range=['#B22222', '#FF6347', '#90EE90'])),  # Colores para las comunas
    tooltip=[
        alt.Tooltip('Comuna', title='Comuna'),
        alt.Tooltip('Ninos_0_15_2021', title='Cantidad de Niños'),
        alt.Tooltip('Tasa_Areas_Verdes', title='Tasa de AAVV por Menor de Edad', format='.2f'),
        alt.Tooltip('Superficie_AAVV_2018', title='Superficie de AAVV (m²)')
    ]
).properties(
title='Gráfico de Dispersión: Niños y Superficie Neta de Áreas Verdes',
    background='white',  # Establecer fondo como transparente
    width=700,  # Ajustar el ancho del gráfico
    height=500  # Ajustar la altura del gráfico
).interactive()

# Ajustar la escala de los ejes para mostrar mejor la relación inversa
x_axis = alt.X('Ninos_0_15_2021:Q', title='Cantidad de Niños')
y_axis = alt.Y('Superficie_AAVV_2018:Q', title='Superficie de AAVV (m²)', scale=alt.Scale(domain=[0, data['Superficie_AAVV_2018'].max()]))

# Combinar el gráfico y ajustar ejes
final_chart = chart.encode(x=x_axis, y=y_axis)
chart.save('chart.html')
# Mostrar el gráfico
final_chart

alt.Chart(...)